In [1]:

url = "https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N"
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

Members = soup.findAll(class_ = 'Members--bio u-richtext')
state_district = soup.findAll(class_ = 'Hero-title')
Money = soup.findAll(class_ = 'Members--number')


#### Get candidate and affiliation info 

In [2]:
# def get_candidate_info(Members):
#     members_list = []
#     member_affiliation = []

#     for i in range(len(Members)):
#         if i == 0:
#             name_affiliation = Members[0].text
#             members_list.append(
#                 re.split(r"[\t\n()]", name_affiliation)[2].strip()
#             )
#             member_affiliation.append(
#                 re.split(r"[\t\n()]", name_affiliation)[3].strip()
#             )
#         else:
            
#             name_affiliation = Members[i].text
#             members_list.append(
#                 re.split(r"[\t()]", name_affiliation)[10].strip()
#                 )
#             member_affiliation.append(
#                 re.split(r"[\t()]", name_affiliation)[11].strip()
#                 )
#     return members_list, member_affiliation

def get_candidate_info(Members):
    members_list = []
    member_affiliation = []

    for i in range(len(Members)):
        members_list.append(
            Members[i].find("h2").text.split('(')[0].split('\t')[-1].strip()
            )
        aff = Members[i].find("h2").text.strip()
        member_affiliation.append(re.split(r"[()]", aff)[1])
    return members_list, member_affiliation


### Return 1 for incumbent = True and 0 if incumbent = False

In [3]:
# def get_incumbent(Members):    
#     list = []
#     for th in Members:
#      #print(th.text.strip())
#         list.append(re.split(r"[\t\n()]", th.text))
#     incumbent_list = []
#     for i in range(len(list)):
#         if any("Incumbent" in item for item in list[i]):
#             incumbent_list.append(1)
#         else:
#             incumbent_list.append(0)
#     return incumbent_list

In [4]:
def get_incumbent(Members):    
    incumbent_list = []
    incumbent_list.append(["Incumbent" in Members[th].text for th in range(0,len(Members))])
    return incumbent_list[0]

#### Get percentage of votes obtained

In [5]:
def get_percentage(Members):
    percentage_list = []
    for th in Members:
        if '%' in th.text:
            percentage_list.append(re.split(r"[()]", th.text)[3].split('%')[0])
        else:
            percentage_list.append(0)
    return percentage_list

In [6]:
# def get_winner(Members):
#     percent_num = []
#     for th in Members:
#         percent_num.append(re.split(r"[(%)]", th.text)[3])
#     winner_list = []
#     for i in range(len(percent_num)):
#         if percent_num[i] == max(percent_num):
#             winner_list.append(1)
#         else:
#             winner_list.append(0)
#     return winner_list

def get_winner(Members):
    winner_list = []
    winner_list.append(["Winner" in Members[th].text for th in range(0,len(Members))])
        
    return winner_list[0]


In [7]:
# def get_state_district(state_district):
#     state_list = []
#     district_list = []
#     for i in range(len(Members)):
#         state_list.append(state_district[0].text.split()[0])
#         district_list.append(state_district[0].text.split()[2])

#     return state_list, district_list

def get_state_district(state_district):
    state_list = []
    district_list = []
    for i in range(len(Members)):
        sd = state_district[0].text
        st = re.split(r"District", sd)[0].strip()
        dis = re.split(r"District",sd)[1].strip().split()[0]
        state_list.append(st)
        district_list.append(dis)

    return state_list, district_list

In [8]:
def get_money_details(Money):
    Money_list = []
    for i in range(len(Money)):
        Money_list.append(float(re.sub(",","", Money[i].text.split("$")[1])))
    Money_raised = Money_list[0::3]
    Money_spent = Money_list[1::3]
    return Money_raised, Money_spent

In [9]:
incumbent_list = get_incumbent(Members)
percentage_list = get_percentage(Members)
winner_list = get_winner(Members)
candidate_list = get_candidate_info(Members)
state_list = get_state_district(state_district)[0]
district_list = get_state_district(state_district)[1]
Money_details = get_money_details(Money)

In [10]:
candidate_df = pd.DataFrame({"Candidate Name" : candidate_list[0],
                             "Affiliation" : candidate_list[1],
                             "Incumbent": incumbent_list,
                             "Percentage of Vote" : percentage_list,
                             "Winner" : winner_list,
                             "State": state_list,
                             "District" : district_list,
                             "Money Raised": Money_details[0],
                             "Money Spent": Money_details[1]}
                             )

In [11]:
candidate_df

,Candidate Name,Affiliation,Incumbent,Percentage of Vote,Winner,State,District,Money Raised,Money Spent
0,Mark Green,R,True,69.9,True,Tennessee,07,1194960.0,935487.0
1,Kiran Sreepada,D,False,27.3,False,Tennessee,07,206644.0,207191.0
2,Ronald Brown,I,False,2.2,False,Tennessee,07,1750.0,0.0
3,Scott Vieira Jr,I,False,0.6,False,Tennessee,07,655.0,1049.0


In [12]:

url_list = []
for i in range(1,10):
    url_list.append(f"https://www.opensecrets.org/races/candidates?cycle=2020&id=TN0{i}&spec=N")
url_list

['https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN02&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN03&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN04&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN05&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN08&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN09&spec=N']

#### Looping through districts 

In [13]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
affiliation_list = []
incumbent_list = []
percentage_list = []
winner_list = []
candidate_list = []
state_list = []
district_list = []
Money_raised = []
Money_spent = []

for url in url_list:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    Members = soup.findAll(class_ = 'Members--bio u-richtext')
    state_district = soup.findAll(class_ = 'Hero-title')
    Money = soup.findAll(class_ = 'Members--number')
    
    incumbent_list.extend(get_incumbent(Members))
    percentage_list.extend(get_percentage(Members))
    winner_list.extend(get_winner(Members))
    candidate_list.extend(get_candidate_info(Members)[0])
    affiliation_list.extend(get_candidate_info(Members)[1])
    state_list.extend(get_state_district(state_district)[0])
    district_list.extend(get_state_district(state_district)[1])
    Money_raised.extend(get_money_details(Money)[0])
    Money_spent.extend(get_money_details(Money)[1])


TN_df = pd.DataFrame({"Candidate Name" : candidate_list,
                             "Affiliation" : affiliation_list,
                             "Incumbent": incumbent_list,
                             "Percentage of Vote" : percentage_list,
                             "Winner" : winner_list,
                             "State": state_list,
                             "District" : district_list,
                             "Money Raised": Money_raised,
                             "Money Spent": Money_spent}
                             )


In [14]:
TN_df['District'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09'], dtype=object)

In [15]:
TN_df["Affiliation"].unique()

array(['R', 'D', 'I'], dtype=object)

### Get the list of Districts of each state

In [16]:

url_ = "https://en.wikipedia.org/wiki/2020_United_States_House_of_Representatives_elections"
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
page_ = requests.get(url_)
soup_ = BeautifulSoup(page_.content, "html.parser")

Districts = soup_.findAll('table',attrs = {'class': 'wikitable', 'class': 'sortable'})
districts_rows = Districts[0].findAll("tr")
districts_rows = districts_rows[2:52]

states_list = []
no_districts = []
for th in districts_rows:
    states_list.append(th.find("th").text.strip())
    no_districts.append(th.find("td").text.strip())

states_df = pd.DataFrame({"State" : states_list,
                             "District" : no_districts}
                             )


In [17]:
states_df['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [18]:
states_abbv = pd.read_csv("../Data/states.csv", delimiter= ",").rename(columns={"Abbreviation" : "State_abbv"})

In [19]:
states_district = states_abbv.merge(states_df, on = "State", how = "inner")
states_district

,State,State_abbv,District
0,Alabama,AL,7
1,Alaska,AK,1
2,Arizona,AZ,9
3,Arkansas,AR,4
4,California,CA,53
5,Colorado,CO,7
6,Connecticut,CT,5
7,Delaware,DE,1
8,Florida,FL,27
9,Georgia,GA,14


### Expand it to all the states and districts

In [20]:
full_url_list = []
for _, row in states_district.iterrows():
    for i in range(1, int(row['District'])+1):
        full_url_list.append(f"https://www.opensecrets.org/races/candidates?cycle=2020&id={row['State_abbv']}{i:02d}&spec=N")


In [21]:
len(full_url_list)

435

In [22]:
full_url_list

['https://www.opensecrets.org/races/candidates?cycle=2020&id=AL01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL02&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL03&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL04&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL05&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL06&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL07&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AK01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ02&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ03&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ04&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ05&spec=N',
 'https://www.opensecrets

In [23]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
affiliation_list = []
incumbent_list = []
percentage_list = []
winner_list = []
candidate_list = []
state_list = []
district_list = []
Money_raised = []
Money_spent = []

for url in full_url_list:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    Members = soup.findAll(class_ = 'Members--bio u-richtext')
    state_district = soup.findAll(class_ = 'Hero-title')
    Money = soup.findAll(class_ = 'Members--number')
    
    incumbent_list.extend(get_incumbent(Members))
    
    percentage_list.extend(get_percentage(Members))
    winner_list.extend(get_winner(Members))
    candidate_list.extend(get_candidate_info(Members)[0])
    affiliation_list.extend(get_candidate_info(Members)[1])
    state_list.extend(get_state_district(state_district)[0])
    district_list.extend(get_state_district(state_district)[1])
    Money_raised.extend(get_money_details(Money)[0])
    Money_spent.extend(get_money_details(Money)[1])


All_data = pd.DataFrame({"Candidate Name" : candidate_list,
                             "Affiliation" : affiliation_list,
                             "Incumbent": incumbent_list,
                             "Percentage of Vote" : percentage_list,
                             "Winner" : winner_list,
                             "State": state_list,
                             "District" : district_list,
                             "Money Raised": Money_raised,
                             "Money Spent": Money_spent}
                             )


In [24]:
All_data.tail(100)

,Candidate Name,Affiliation,Incumbent,Percentage of Vote,Winner,State,District,Money Raised,Money Spent
792,Henry Cuellar,D,True,58.3,True,Texas,28,2655317.0,3650848.0
793,Sandra Whitten,R,False,39.0,False,Texas,28,24760.0,27354.0
794,Sylvia Garcia,D,True,71.1,True,Texas,29,818984.0,758382.0
795,Jaimy Blanco,R,False,27.4,False,Texas,29,4378.0,0.0
796,Eddie Bernice Johnson,D,True,77.5,True,Texas,30,589096.0,494547.0
...,...,...,...,...,...,...,...,...,...
887,Tricia Zunker,D,False,39.2,False,Wisconsin,07,1261957.0,1232690.0
888,Mike Gallagher,R,True,64.0,True,Wisconsin,08,3202905.0,2841801.0
889,Amanda Stuck,D,False,36.0,False,Wisconsin,08,416978.0,399916.0
890,Liz Cheney,R,True,68.6,True,Wyoming,01,3003883.0,3060167.0


In [25]:
All_data.to_csv("../Data/all_data.csv", index=False)